# Steps in this project
1. Authenticate user
2. Call the Docs API
3. Store input
4. Create Google Doc
5. Paste the parsed content into the document
    1. Follow the formatting rules programmatically


In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
# Step 1: Authenticate
from google.colab import auth
auth.authenticate_user()

In [3]:
# Step 2: Create Docs service
import google.auth
from googleapiclient.discovery import build

creds, _ = google.auth.default()
service = build('docs', 'v1', credentials=creds)

In [4]:
# Step 3: Define your markdown notes
markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [5]:
# Step 4: Create Google Doc
doc_title = "Product Team Sync - May 15, 2023"
doc = service.documents().create(body={'title': doc_title}).execute()
doc_id = doc['documentId']

In [6]:
# This dumps data into the Google Doc

# Step 5: Insert markdown as plain text
# service.documents().batchUpdate(
#     documentId=doc_id,
#     body={'requests': [
#         {'insertText': {'location': {'index': 1}, 'text': markdown_notes}}
#     ]}
# ).execute()

# print(f"Google Doc created successfully! Document ID: {doc_id}")
# print(f"Open it here: https://docs.google.com/document/d/{doc_id}/edit")


In [7]:
#Took help of AI for the formatting aspect


def format_meeting_notes(service, doc_id, markdown_notes):
    """
    Inserts markdown notes into Google Docs with:
    - Heading 1 for main title
    - Heading 2 for sections
    - Heading 3 for sub-sections
    """
    lines = markdown_notes.split("\n")
    requests = []
    index_pointer = 1  # start at beginning of doc

    for line in lines:
        stripped = line.strip()
        if not stripped:
            text = "\n"
            requests.append({'insertText': {'location': {'index': index_pointer}, 'text': text}})
            index_pointer += len(text)
            continue

        # Heading 1: main title
        if stripped.startswith("# "):
            text = stripped[2:] + "\n"
            requests.append({'insertText': {'location': {'index': index_pointer}, 'text': text}})
            requests.append({'updateParagraphStyle': {
                'range': {'startIndex': index_pointer, 'endIndex': index_pointer + len(text)},
                'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                'fields': 'namedStyleType'
            }})
            index_pointer += len(text)

        # Heading 2: sections
        elif stripped.startswith("## "):
            text = stripped[3:] + "\n"
            requests.append({'insertText': {'location': {'index': index_pointer}, 'text': text}})
            requests.append({'updateParagraphStyle': {
                'range': {'startIndex': index_pointer, 'endIndex': index_pointer + len(text)},
                'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                'fields': 'namedStyleType'
            }})
            index_pointer += len(text)

        # Heading 3: sub-sections
        elif stripped.startswith("### "):
            text = stripped[4:] + "\n"
            requests.append({'insertText': {'location': {'index': index_pointer}, 'text': text}})
            requests.append({'updateParagraphStyle': {
                'range': {'startIndex': index_pointer, 'endIndex': index_pointer + len(text)},
                'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                'fields': 'namedStyleType'
            }})
            index_pointer += len(text)

        # Normal text
        else:
            text = stripped + "\n"
            requests.append({'insertText': {'location': {'index': index_pointer}, 'text': text}})
            index_pointer += len(text)

    # Execute all requests
    try:
        service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
        print(f"Google Doc formatted successfully! Open it here: https://docs.google.com/document/d/{doc_id}/edit")
    except HttpError as e:
        print("Error formatting document:", e)


# Call the function
format_meeting_notes(service, doc_id, markdown_notes)


Google Doc formatted successfully! Open it here: https://docs.google.com/document/d/17ayMT7E0ByWuC1Z1rhFYP5Db9dPSsofQxUxbAGhSmII/edit
